In [12]:
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.datasets import mnist
import itertools
import random

In [13]:
# Randomly select a defined number of image paires from a given dataset
def get_left_min_image_combinations(number_image_pairs_selected, image_data, image_labels):
    
    num_images_total = len(image_labels)
    
    image_combination_data = []
    image_combination_labels = []
    
    for i in range(number_image_pairs_selected):
        
        # Draw two image indices from a uniform random distribution
        random_index_A = random.randint(0, num_images_total-1)
        random_index_B = random.randint(0, num_images_total-1)
        
        # Randomly choose two images from the dataset
        image_A = image_data[random_index_A]
        image_B = image_data[random_index_B]
        
        # Merge images TODO: this could be a function with several options
        combined_images = np.hstack((image_A, image_B))
        
        # Find the minimum between the two labels
        label_A = image_labels[random_index_A]
        label_B = image_labels[random_index_B]
        minimum_label = min(label_A, label_B)
        
        if label_A < label_B:
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_A, image_B))
        elif label_A > label_B: 
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_B, image_A))
        # Here we want to exclude the images where they are the same on both sides
            
   
        # Append newly generated image combination and minimum label to list
        image_combination_data.append(combined_images)
        image_combination_labels.append(minimum_label)
        
    # Convert image data and labels lists to numpy arrays
    image_combination_data = np.array(image_combination_data)
    image_combination_labels = np.array(image_combination_labels)
    
    return image_combination_data, image_combination_labels

In [14]:
# Randomly select a defined number of image paires from a given dataset
def get_right_min_image_combinations(number_image_pairs_selected, image_data, image_labels):
    
    num_images_total = len(image_labels)
    
    image_combination_data = []
    image_combination_labels = []
    
    for i in range(number_image_pairs_selected):
        
        # Draw two image indices from a uniform random distribution
        random_index_A = random.randint(0, num_images_total-1)
        random_index_B = random.randint(0, num_images_total-1)
        
        # Randomly choose two images from the dataset
        image_A = image_data[random_index_A]
        image_B = image_data[random_index_B]
        
        # Merge images TODO: this could be a function with several options
        combined_images = np.hstack((image_A, image_B))
        
        # Find the minimum between the two labels
        label_A = image_labels[random_index_A]
        label_B = image_labels[random_index_B]
        minimum_label = min(label_A, label_B)
        
        if label_A < label_B:
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_B, image_A))
        elif label_A > label_B: 
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_A, image_B))
        # Here we want to exclude the images where they are the same on both sides
            
   
        # Append newly generated image combination and minimum label to list
        image_combination_data.append(combined_images)
        image_combination_labels.append(minimum_label)
        
    # Convert image data and labels lists to numpy arrays
    image_combination_data = np.array(image_combination_data)
    image_combination_labels = np.array(image_combination_labels)
    
    return image_combination_data, image_combination_labels

In [15]:
# Load data and split into training and test
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [16]:
# Split training set further into training and validation sets
sample_size = 40000
X_train, X_val, y_train, y_val = train_images[:sample_size], train_images[sample_size:sample_size*2],  train_labels[:sample_size], train_labels[sample_size:sample_size*2]

In [17]:
combo_train_X, combo_train_y = get_left_min_image_combinations(40000, X_train, y_train)
combo_val_X, combo_val_y = get_right_min_image_combinations(20000, X_val, y_val)

In [18]:
# Reshape the data so they can be fed into the network, one-hot-encode the labels

train_images = combo_train_X.reshape((combo_train_X.shape[0], 28, 56, 1))
train_images = train_images/ 255

val_images = combo_val_X.reshape((combo_val_X.shape[0], 28, 56, 1))
val_images = val_images/255

train_labels = to_categorical(combo_train_y)
val_labels = to_categorical(combo_val_y)

In [19]:
# Model architecture
def build_CNN():

    # Convolutional NN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,56,1)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    # Adding a NN Classifier
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))

    model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy', 'mae'])

    return model

In [22]:
CNN_model = build_CNN()
CNN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [23]:
# Fit model
CNN_model.fit(train_images, 
          train_labels, 
          epochs = 5, 
          batch_size = 64)

Epoch 1/5
625/625 [==============================] - 50s 78ms/step - loss: 0.1787 - accuracy: 0.9426 - mae: 0.0171
Epoch 2/5
625/625 [==============================] - 48s 77ms/step - loss: 0.0359 - accuracy: 0.9888 - mae: 0.0034
Epoch 3/5
625/625 [==============================] - 48s 77ms/step - loss: 0.0213 - accuracy: 0.9937 - mae: 0.0019
Epoch 4/5
625/625 [==============================] - 47s 76ms/step - loss: 0.0145 - accuracy: 0.9953 - mae: 0.0014
Epoch 5/5
625/625 [==============================] - 47s 76ms/step - loss: 0.0103 - accuracy: 0.9966 - mae: 8.9298e-04


In [25]:
# Print loss, mae and accuracy
eval_model = CNN_model.evaluate(val_images, val_labels)
eval_model

625/625 [==============================] - 5s 7ms/step - loss: 19.6199 - accuracy: 0.1087 - mae: 0.1780


[19.619888305664062, 0.10869999974966049, 0.17800095677375793]

In [26]:
# Model architecture
def build_DNN():

    # Convolutional NN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,56,1)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    # Adding a NN Classifier
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'softmax'))

    # Compile model
    model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy', 'mae'])

    return model

In [27]:
DNN_model = build_CNN()
DNN_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)               

In [28]:
# Fit model
DNN_model.fit(train_images, 
          train_labels, 
          epochs = 5, 
          batch_size = 64)

Epoch 1/5
625/625 [==============================] - 49s 76ms/step - loss: 0.1911 - accuracy: 0.9405 - mae: 0.0184
Epoch 2/5
625/625 [==============================] - 48s 76ms/step - loss: 0.0378 - accuracy: 0.9884 - mae: 0.0036
Epoch 3/5
625/625 [==============================] - 46s 73ms/step - loss: 0.0213 - accuracy: 0.9940 - mae: 0.0020
Epoch 4/5
625/625 [==============================] - 46s 74ms/step - loss: 0.0143 - accuracy: 0.9955 - mae: 0.0013
Epoch 5/5
625/625 [==============================] - 48s 76ms/step - loss: 0.0099 - accuracy: 0.9969 - mae: 8.6811e-04


In [29]:
# Print loss, mae and accuracy
eval_model = DNN_model.evaluate(val_images, val_labels)
eval_model

625/625 [==============================] - 5s 8ms/step - loss: 19.0342 - accuracy: 0.1058 - mae: 0.1786


[19.034225463867188, 0.10575000196695328, 0.1786217838525772]